<a href="https://colab.research.google.com/github/Garry1970/Garry1970/blob/main/%D0%92%D0%BE%D0%B5%D0%BD%D0%BD%D1%8B%D0%B5_%D1%80%D0%B0%D1%81%D1%85%D0%BE%D0%B4%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
"""
Перед запуском установите зависимости:
pip install pandas plotly pycountry requests
"""

import pandas as pd
import plotly.express as px
import pycountry
import requests
from zipfile import ZipFile
import io

# ----------------------------
# 1. Загрузка данных из Всемирного банка
# ----------------------------

def load_wb_data(indicator_code, year=2022):
    """Загрузка данных по коду индикатора"""
    url = f"https://api.worldbank.org/v2/en/indicator/{indicator_code}?downloadformat=csv"
    try:
        response = requests.get(url, timeout=10)
        zip_file = ZipFile(io.BytesIO(response.content))

        # Поиск файла данных
        csv_files = [f for f in zip_file.namelist() if f.startswith("API_") and "Metadata" not in f]
        if not csv_files:
            raise ValueError("CSV файл не найден")

        df = pd.read_csv(zip_file.open(csv_files[0]), skiprows=3)
        df = df[['Country Name', str(year)]].copy()
        df.columns = ['country', 'value']
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        return df.dropna()
    except Exception as e:
        print(f"Ошибка загрузки {indicator_code}: {str(e)}")
        return pd.DataFrame()

# Загрузка данных за 2022 год
df_milex = load_wb_data("MS.MIL.XPND.CD")  # Военные расходы в USD
df_gdp = load_wb_data("NY.GDP.MKTP.CD")    # ВВП в USD

# ----------------------------
# 2. Обработка данных
# ----------------------------

# Словарь для ручного сопоставления стран
country_name_map = {
    "United States": "United States of America",
    "Russia": "Russian Federation",
    "Iran": "Iran, Islamic Republic of",
    "South Korea": "Korea, Republic of",
    "Egypt, Arab Rep.": "Egypt",
    "Venezuela, RB": "Venezuela, Bolivarian Republic of",
    "Syrian Arab Republic": "Syria",
    "Congo, Dem. Rep.": "Congo, The Democratic Republic of the",
    "Yemen, Rep.": "Yemen"
}

def get_iso_code(country_name):
    """Конвертация названий в ISO-коды"""
    try:
        country_name = country_name_map.get(country_name, country_name)
        return pycountry.countries.search_fuzzy(country_name)[0].alpha_3
    except (LookupError, AttributeError):
        print(f"Страна не найдена: {country_name}")
        return None

# Объединение данных
if not df_milex.empty and not df_gdp.empty:
    df = pd.merge(
        df_milex.rename(columns={'value': 'military_expenditure'}),
        df_gdp.rename(columns={'value': 'gdp'}),
        on='country',
        how='inner'
    )

    # Расчет доли военных расходов
    df['military_share'] = (df['military_expenditure'] / df['gdp'] * 100).round(2)
    df['iso_code'] = df['country'].apply(get_iso_code)
    df = df.dropna(subset=['iso_code'])
else:
    print("Ошибка: Не удалось загрузить данные")
    df = pd.DataFrame()

# ----------------------------
# 3. Визуализация
# ----------------------------

if not df.empty:
    fig = px.choropleth(
        df,
        locations="iso_code",
        color="military_share",
        hover_name="country",
        hover_data={
            'military_expenditure': '$,.0f',
            'gdp': '$,.0f',
            'military_share': '.2f%',
            'iso_code': False
        },
        color_continuous_scale=px.colors.sequential.Reds,
        title="Доля военных расходов от ВВП (2022, Всемирный банк)",
        labels={'military_share': 'Доля ВПК, %'},
        projection="natural earth"
    )

    fig.update_layout(
        margin={"r":0, "t":40, "l":0, "b":0},
        coloraxis_colorbar={
            'title': 'Доля ВПК, %',
            'thickness': 15,
            'len': 0.75
        }
    )

    fig.show()
else:
    print("Нет данных для визуализации")

Страна не найдена: Africa Eastern and Southern
Страна не найдена: Africa Western and Central
Страна не найдена: Arab World
Страна не найдена: Central Europe and the Baltics
Страна не найдена: Congo, Rep.
Страна не найдена: East Asia & Pacific (excluding high income)
Страна не найдена: Early-demographic dividend
Страна не найдена: East Asia & Pacific
Страна не найдена: Europe & Central Asia (excluding high income)
Страна не найдена: Europe & Central Asia
Страна не найдена: Euro area
Страна не найдена: European Union
Страна не найдена: Fragile and conflict affected situations
Страна не найдена: Gambia, The
Страна не найдена: High income
Страна не найдена: Heavily indebted poor countries (HIPC)
Страна не найдена: IBRD only
Страна не найдена: IDA & IBRD total
Страна не найдена: IDA total
Страна не найдена: IDA blend
Страна не найдена: IDA only
Страна не найдена: Iran, Islamic Rep.
Страна не найдена: Korea, Rep.
Страна не найдена: Latin America & Caribbean (excluding high income)
Страна не 